In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import gzip

# Loading voting data

In [2]:
DATA_FOLDER = "../data/"
GENERATED_FOLDER = "../generated/"

In [3]:
SOURCE_VOTE_DATA_DTYPES = {
    'AffairShortId': np.int, 
    'AffairTitle': np.str,
    'VoteRegistrationNumber': np.int, 
    'VoteDate': np.str,
    'VoteMeaningYes': np.str,
    'VoteMeaningNo': np.str, 
    'DivisionText': np.str, 
    'VoteSubmissionText': np.str,
    'VoteFilteredYes': np.uint8, 
    'VoteFilteredNo': np.uint8, 
    'VoteFilteredAbstain': np.uint8,
    'VoteFilteredNotParticipated': np.uint8, 
    'VoteFilteredExcused': np.uint8,
    'VoteFilteredPresident': np.uint8, 
    'CouncillorId': np.int, 
    'CouncillorName': np.str,
    'CouncillorYes': np.uint0, 
    'CouncillorNo': np.uint0, 
    'CouncillorAbstain': np.uint0,
    'CouncillorNotParticipated': np.uint0, 
    'CouncillorExcused': np.uint0,
    'CouncillorPresident': np.uint0
}

We need some custom functions because the source file is not properly formatted.

In [4]:
def read_file(name):
    path = DATA_FOLDER + name + ".csv.gz"
    with gzip.open(path, 'rt') as file:
        for line in file:
            columns = line.split('","')
            yield [column.strip(u'"\n\ufeff') for column in columns]

def load(name):
    rows = np.array(list(read_file(name)))
    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

def load_all():
    dfs = []
    for year in range(2007, 2020):
        dfs.append(load(str(year)))
    df = pd.concat(dfs, ignore_index=True, copy=False).astype(SOURCE_VOTE_DATA_DTYPES)
    return df

In [5]:
vote_data = load_all()

## Formatting Dates

In [6]:
SOURCE_VOTE_DATA_DATE_FORMAT = '%a %b %d %Y %H:%M:%S %Z'

In [7]:
vote_data['VoteDate'] = pd.to_datetime(vote_data['VoteDate'].str[:28], format=SOURCE_VOTE_DATA_DATE_FORMAT)

## Saving Dataframe

In [8]:
vote_data.to_csv(GENERATED_FOLDER + 'vote_data.csv.gz')

## Reading Dataframe

In [9]:
VOTE_DATA_DTYPES = {
    'AffairShortId': np.int, 
    'AffairTitle': np.str,
    'VoteRegistrationNumber': np.int, 
    'VoteMeaningYes': np.str,
    'VoteMeaningNo': np.str, 
    'DivisionText': np.str, 
    'VoteSubmissionText': np.str,
    'VoteFilteredYes': np.uint8, 
    'VoteFilteredNo': np.uint8, 
    'VoteFilteredAbstain': np.uint8,
    'VoteFilteredNotParticipated': np.uint8, 
    'VoteFilteredExcused': np.uint8,
    'VoteFilteredPresident': np.uint8, 
    'CouncillorId': np.int, 
    'CouncillorName': np.str,
    'CouncillorYes': np.uint0, 
    'CouncillorNo': np.uint0, 
    'CouncillorAbstain': np.uint0,
    'CouncillorNotParticipated': np.uint0, 
    'CouncillorExcused': np.uint0,
    'CouncillorPresident': np.uint0
}

VOTE_DATA_DATE_COLUMNS = ['VoteDate']

In [10]:
vote_data = pd.read_csv(GENERATED_FOLDER + 'vote_data.csv.gz', dtype=VOTE_DATA_DTYPES, parse_dates=VOTE_DATA_DATE_COLUMNS, date_parser=dt.datetime.fromisoformat)
vote_data.head()

,Unnamed: 0,AffairShortId,AffairTitle,VoteRegistrationNumber,VoteDate,VoteMeaningYes,VoteMeaningNo,DivisionText,VoteSubmissionText,VoteFilteredYes,...,VoteFilteredExcused,VoteFilteredPresident,CouncillorId,CouncillorName,CouncillorYes,CouncillorNo,CouncillorAbstain,CouncillorNotParticipated,CouncillorExcused,CouncillorPresident
0,0,20070464,Prorogation de la loi fédérale sur l'adaptatio...,248,2007-12-21 09:50:38+00:00,NaN,NaN,Ja,Vote final,196,...,0,1,3923,Marra Ada,1,0,0,0,0,0
1,1,20070464,Prorogation de la loi fédérale sur l'adaptatio...,248,2007-12-21 09:50:38+00:00,NaN,NaN,Ja,Vote final,196,...,0,1,3883,Glauser-Zufferey Alice,1,0,0,0,0,0
2,2,20070464,Prorogation de la loi fédérale sur l'adaptatio...,248,2007-12-21 09:50:38+00:00,NaN,NaN,Ja,Vote final,196,...,0,1,3907,Thorens Goumaz Adèle,1,0,0,0,0,0
3,3,20070464,Prorogation de la loi fédérale sur l'adaptatio...,248,2007-12-21 09:50:38+00:00,NaN,NaN,Ja,Vote final,196,...,0,1,3913,Wyss Brigit,1,0,0,0,0,0
4,4,20070464,Prorogation de la loi fédérale sur l'adaptatio...,248,2007-12-21 09:50:38+00:00,NaN,NaN,Ja,Vote final,196,...,0,1,3878,Flückiger-Bäni Sylvia,1,0,0,0,0,0


# Creating 'Votes' Dataframe
The votes dataframe should have the VoteId as index and a column for each councillor with the values being the vote from the respective councillor.
Votes have 6 possible values:

- 'Yes'
- 'No'
- 'Abstain'
- 'NotParticipated'
- 'Excused'
- 'President'

We use integer ids to encode these values as:

- 'Yes': 0
- 'No': 1
- 'Abstain': 2
- 'NotParticipated': 3
- 'Excused': 4
- 'President': 5

Missing values (the councillor was not in the national council when the vote happend) are encoded as -1.

Test to make sure that no data is missing and our assumptions that at exactly one of the possible values is present:

In [11]:
vote_data.loc[:, 'CouncillorYes':'CouncillorPresident'].sum(axis=1).describe()

count    2711799.0
mean           1.0
std            0.0
min            1.0
25%            1.0
50%            1.0
75%            1.0
max            1.0
dtype: float64

We see that max=1 and min=1, so there is exactly one value true in the columns.

Here we create the votes dataframe via a temporary dataframe.

In [12]:
temp = vote_data[['AffairShortId', 'CouncillorId']].copy()
temp['VoteMeaning'] = -1
temp.loc[vote_data['CouncillorYes'] == 1, 'VoteMeaning'] = 0
temp.loc[vote_data['CouncillorNo'] == 1, 'VoteMeaning'] = 1
temp.loc[vote_data['CouncillorAbstain'] == 1, 'VoteMeaning'] = 2
temp.loc[vote_data['CouncillorNotParticipated'] == 1, 'VoteMeaning'] = 3
temp.loc[vote_data['CouncillorExcused'] == 1, 'VoteMeaning'] = 4
temp.loc[vote_data['CouncillorPresident'] == 1, 'VoteMeaning'] = 5
votes = temp.groupby(['AffairShortId', 'CouncillorId']).aggregate('first').unstack()
del temp

In [13]:
votes.columns = votes.columns.get_level_values(1)
votes = votes.fillna(-1).astype(int)
votes.to_csv(GENERATED_FOLDER + 'votes.csv.gz')

In [14]:
votes = pd.read_csv(GENERATED_FOLDER + 'votes.csv.gz', index_col='AffairShortId')

In [15]:
votes.head()

,15,21,26,28,34,61,70,74,76,91,...,4224,4225,4226,4227,4228,4229,4230,4232,4234,4236
AffairShortId,,,,,,,,,,,,,,,,,,,,,
1,0,3,0,0,-1,-1,-1,0,-1,0,...,1,0,1,1,1,0,1,1,0,0
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,...,-1,3,-1,-1,-1,-1,-1,-1,-1,-1
20000421,0,-1,0,0,0,-1,0,-1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
20000431,0,-1,0,0,3,-1,0,-1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
20000436,0,-1,0,0,-1,-1,-1,-1,-1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


# Loading Members Data

In [16]:
SOURCE_MEMBERS_DTYPES = {
    'Active': np.bool,
    'FirstName': np.str,
    'LastName': np.str,
    'GenderAsString': np.str,
    'CantonName': np.str,
    'CantonAbbreviation': np.str,
    'CouncilName': np.str,
    'ParlGroupName': np.str,
    'ParlGroupAbbreviation': np.str,
    'PartyName': np.str,
    'PartyAbbreviation': np.str,
    'MaritalStatusText': np.str,
    'BirthPlace_City': np.str,
    'BirthPlace_Canton': np.str,
    'Mandates': np.str,
    'Citizenship': np.str,
    'CouncillorName': np.str
}

SOURCE_MEMBERS_DATE_COLUMNS = ['DateJoining', 'DateLeaving', 'DateOfBirth', 'DateOfDeath']
SOURCE_MEMBERS_DATE_FORMAT = "%M/%d/%Y"

In [17]:
source_members = pd.read_excel(DATA_FOLDER + 'Ratsmitglieder_1848_EN.xlsx', dtype=SOURCE_MEMBERS_DTYPES, parse_dates=SOURCE_MEMBERS_DATE_COLUMNS, date_format=SOURCE_MEMBERS_DATE_FORMAT)
for column in SOURCE_MEMBERS_DATE_COLUMNS:
    source_members[column] = source_members[column].dt.date
source_members.head()

,Active,FirstName,LastName,GenderAsString,CantonName,CantonAbbreviation,CouncilName,ParlGroupName,ParlGroupAbbreviation,PartyName,PartyAbbreviation,MaritalStatusText,BirthPlace_City,BirthPlace_Canton,Mandates,DateJoining,DateLeaving,Citizenship,DateOfBirth,DateOfDeath
0,False,Giuseppe,a Marca,m,Grisons,GR,Conseil des Etats,Centre,MC,Conservateurs,Cons*,NaN,NaN,NaN,NaN,1849-12-01,1851-07-01,Soazza (GR),1799-07-29,1866-07-16
1,False,Alois,Ab Yberg,m,Schwyz,SZ,Conseil national,Groupe radical-démocratique,R,Parti radical-démocratique suisse,PRD,NaN,NaN,NaN,NaN,1928-12-03,1935-12-01,Schwyz (SZ),1878-10-06,1959-10-17
2,False,Fabio,Abate,m,Tessin,TI,Conseil national,Groupe radical-libéral,RL,Parti radical-démocratique suisse,PRD,NaN,Locarno,Tessin,Incarichi esecutivi presso il comune di Locarn...,2007-12-03,2011-12-04,Cabbio (TI),1966-01-04,NaT
3,False,Fabio,Abate,m,Tessin,TI,Conseil national,Groupe radical-libéral,R,Parti radical-démocratique suisse,PRD,NaN,Locarno,Tessin,Incarichi esecutivi presso il comune di Locarn...,2000-09-25,2003-11-30,Cabbio (TI),1966-01-04,NaT
4,False,Fabio,Abate,m,Tessin,TI,Conseil des Etats,Groupe libéral-radical,RL,PLR.Les Libéraux-Radicaux,PLR,NaN,Locarno,Tessin,Incarichi esecutivi presso il comune di Locarn...,2011-12-05,2015-11-29,Cabbio (TI),1966-01-04,NaT


We want to remove all councillors that did not participate in any votes in the votes dataframe. We only want councillors in the national council and members that were active during the period in question. First we check whether there is any data missing:

In [18]:
source_members.isna().any()

Active                   False
FirstName                False
LastName                 False
GenderAsString           False
CantonName                True
CantonAbbreviation        True
CouncilName               True
ParlGroupName             True
ParlGroupAbbreviation     True
PartyName                 True
PartyAbbreviation         True
MaritalStatusText         True
BirthPlace_City           True
BirthPlace_Canton         True
Mandates                  True
DateJoining              False
DateLeaving               True
Citizenship               True
DateOfBirth              False
DateOfDeath               True
dtype: bool

We have all the data for when the people joined but some data is missing in the 'DateLeaving' column. It turns out that all entries with missing 'DateLeaving' values have a 'DateJoining' value after 2007. So we want to keep them. See next cell for analysis:

In [19]:
source_members.loc[source_members['DateLeaving'].isna(), 'DateJoining'].min()

datetime.date(2015, 11, 30)

We can now discard all entries for councillors which left before the first vote in the other dataset:

In [20]:
did_not_leave = source_members['DateLeaving'].isna()
left_after_first_vote = source_members['DateLeaving'] > vote_data['VoteDate'].dt.date.min()

In [21]:
members = source_members[did_not_leave | left_after_first_vote]

In [22]:
members.to_csv(GENERATED_FOLDER + 'members.csv', index=False)

In [23]:
MEMBERS_DTYPES = {
    'Active': np.bool,
    'FirstName': np.str,
    'LastName': np.str,
    'GenderAsString': np.str,
    'CantonName': np.str,
    'CantonAbbreviation': np.str,
    'CouncilName': np.str,
    'ParlGroupName': np.str,
    'ParlGroupAbbreviation': np.str,
    'PartyName': np.str,
    'PartyAbbreviation': np.str,
    'MaritalStatusText': np.str,
    'BirthPlace_City': np.str,
    'BirthPlace_Canton': np.str,
    'Mandates': np.str,
    'Citizenship': np.str,
    'CouncillorName': np.str
}

MEMBERS_DATE_COLUMNS = ['DateJoining', 'DateLeaving', 'DateOfBirth', 'DateOfDeath']

In [24]:
members = pd.read_csv(GENERATED_FOLDER + 'members.csv', parse_dates=MEMBERS_DATE_COLUMNS)
for column in MEMBERS_DATE_COLUMNS:
    members[column] = members[column].dt.date